In [14]:
import rotalysis.pump.curve_generator as cg
from rotalysis.pump.pump import Pump

In [15]:
rated_flow = 550
rated_head = 470

In [16]:
(a, b, c) = cg.get_headcurve_coeff_from_twopoint(
    rated_flow,
    rated_head,
)

In [17]:
test_flow = [rated_flow * i / 10 for i in range(13)]
test_heads = [cg.get_head_from_curve(x, a, b, c) for x in test_flow]

In [19]:
from rotalysis.pump.pump_function import PumpFunction as pf

In [20]:
pf.get_pump_efficiency(rated_flow, 0.70, 500)

0.6942148760330579